In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Загрузка данных из URL
url = "https://storage.yandexcloud.net/academy.ai/SBER.csv"
df = pd.read_csv(url, sep=';')

# Преобразование столбца DATE в формат datetime
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

# Параметры для расчёта
ma_size = 20  # Период для SMA (20 дней)
bol_size = 2  # Ширина коридора (стандартное отклонение * 2)

# Расчёт SMA (простая скользящая средняя)
df['SMA'] = df['CLOSE'].rolling(ma_size).mean().shift()

# Расчёт верхней и нижней полос Боллинджера
df['BB_UP'] = df['SMA'] + df['CLOSE'].rolling(ma_size).std() * bol_size
df['BB_DOWN'] = df['SMA'] - df['CLOSE'].rolling(ma_size).std() * bol_size

# Создание графика свечей
fig = go.Figure(go.Candlestick(
    x=df['DATE'],
    open=df['OPEN'],
    high=df['HIGH'],
    low=df['LOW'],
    close=df['CLOSE'],
    name='Цена'
))

# Добавление SMA (средняя линия)
fig.add_trace(go.Scatter(
    x=df.DATE,
    y=df.SMA,
    name='SMA (20 дней)',
    line_width=1,
    line_color="#ff7738"
))

# Добавление верхней полосы Боллинджера
fig.add_trace(go.Scatter(
    x=df.DATE,
    y=df.BB_UP,
    line_width=1,
    line_color="#0000ff",
    showlegend=False
))

# Добавление нижней полосы Боллинджера
fig.add_trace(go.Scatter(
    x=df.DATE,
    y=df.BB_DOWN,
    line_width=1,
    line_color="#0000ff",
    showlegend=False
))

# Настройка графика
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(
    title='Стоимость акций Сбербанка с полосами Боллинджера',
    yaxis_title='Цена (руб)',
    xaxis_title='Дата'
)

# Добавление аннотаций для примеров торговли
fig.update_layout(
    annotations=[
        # Пример 1: Покупка
        dict(
            x='2017-01-27', y=179, xanchor='right',
            text='Покупка', bgcolor='yellowgreen', font={'size': 20},
            showarrow=True, arrowhead=3, arrowsize=2
        ),
        # Пример 1: Продажа
        dict(
            x='2017-02-09', y=170, xanchor='right',
            text='Продажа', bgcolor='yellow', font={'size': 20},
            showarrow=True, arrowhead=3, arrowsize=2,
            axref='x domain', ax=-30, ayref='y domain', ay=30
        ),
        # Пример 2: Покупка
        dict(
            x='2019-10-02', y=223, xanchor='right',
            text='Покупка', bgcolor='yellowgreen', font={'size': 20},
            showarrow=True, arrowhead=3, arrowsize=2,
            axref='x domain', ax=-30, ayref='y domain', ay=30
        ),
        # Пример 2: Продажа
        dict(
            x='2019-10-11', y=230, xanchor='left',
            text='Продажа', bgcolor='yellow', font={'size': 20},
            showarrow=True, arrowhead=3, arrowsize=2,
            axref='x domain', ax=30, ayref='y domain', ay=30
        )
    ]
)

# Отображение графика
fig.show()

In [ ]:
# Анализ графика:
# SMA (20 дней): Это средняя цена за последние 20 дней. Она помогает определить общий тренд.
# Верхняя полоса (BB_UP): Показывает уровень перекупленности. Если цена приближается к этой линии, это может сигнализировать о возможном снижении цены.
# Нижняя полоса (BB_DOWN): Показывает уровень перепроданности. Если цена приближается к этой линии, это может сигнализировать о возможном росте цены.

# Торговая стратегия:
# На основе полос Боллинджера можно предложить следующую стратегию:
# Покупка (вход в позицию):
# Когда цена закрытия пересекает нижнюю полосу Боллинджера (BB_DOWN) снизу вверх, это может быть сигналом к покупке.
# Продажа (выход из позиции):
# Когда цена закрытия пересекает верхнюю полосу Боллинджера (BB_UP) сверху вниз, это может быть сигналом к продаже.